<a href="https://colab.research.google.com/github/SruthiM-10/Irrigation-Scheduling-Recommendations-And-Dataset/blob/main/dataset_processing/(1)_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openai import OpenAI
import re
import json
import requests
import os

In [ ]:
! pip install -q langchain_community langchain_openai faiss-cpu langchain langchain_core langgraph pinecone PyPDF2 doi2pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/20

In [ ]:
import time
import uuid
import requests
from langchain_community.document_loaders import WebBaseLoader
from typing import Dict, List, Any, Tuple, Optional, Union
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
import pickle
from langchain_core.pydantic_v1 import Field, conlist, BaseModel

import warnings

warnings.filterwarnings(
    "ignore",
    category=UserWarning,
    module='langchain_openai.chat_models.base'
)

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Data Per Feature (Test 1)

In [ ]:
all_samples = pd.read_csv('/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/all_feature_samples.csv')

In [ ]:
all_samples.loc[all_samples["content source"] == "pdf"]

,Unnamed: 0,irrigation_scheduling_method,sensor_type,sensor_threshold,number_of_sensors,evapotranspiration_level,daily_drainage,amount_irrigated,number_of_irrigation_events,irrigation_time,...,plant_height,water_productivity,doi_url,file_path,doi,references,processed_at,content source,relevance_score,relevance_dict
0,0,Soil moisture sensor-based method (T-VWC),Soil moisture sensor,VWC max = 0.507 m3 m−3,1,ETd = 4.63-4.66 mm,0.34-0.43 L plant−1,1.70-2.00 L plant−1,12,07:00-18:00,...,NaN,17.37 kg m−3,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
1,1,Accumulated radiation combined with soil moist...,Soil moisture sensor,Accumulated radiation = 0.8 MJ m−2,1,ETd = 4.66 mm,0.18-0.40 L plant−1,0.85-2.03 L plant−1,14,NaN,...,NaN,20.52 kg m−3,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
2,2,Crop evapotranspiration estimated method using...,NaN,ETc h ≥ 0.4 mm,NaN,ETd = 2.94 mm,0.26-0.50 L plant−1,1.12-1.41 L plant−1,9-10,NaN,...,NaN,25.07 kg m−3,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,seed-local:fad17c8d-409c-4b61-9cec-3aa99fa251a6,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
3,3,Deficit irrigation,"neutron probe, gravimetric method",50% depletion of total available soil moisture,NaN,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc",NaN,"395 mm (2021/2022), 305 mm (2022/2023)","14 (2021/2022), 11 (2022/2023)",December to March,...,NaN,1.42 kg m−3 (70% ETc),seed-local:e3c05b96-64d7-41e5-9bb9-04c829904fd4,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,seed-local:e3c05b96-64d7-41e5-9bb9-04c829904fd4,[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
4,4,Deficit irrigation,"neutron probe, gravimetric method",50% depletion of total available soil moisture,NaN,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc",NaN,"417.2 mm (100% ETc), 341 mm (85% ETc), 286 mm ...","14 (2021/2022), 11 (2022/2023)",December to March (2021/2022 and 2022/2023),...,NaN,1.42 kg m−3 (70% ETc),https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:02:30,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542,542,Maximum Allowable Depletion (MAD) 60%,"Soil Watch 10, PHYTOS 31",60% MAD,3,NaN,NaN,"19 mm for corn, 13 mm for soybeans",1-8,During critical phases,...,NaN,NaN,seed-local:fdf0162d-8971-4c8f-b80a-68d0af3ab55e,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,seed-local:fdf0162d-8971-4c8f-b80a-68d0af3ab55e,[{'title': 'Understanding the impact of irriga...,2025-09-12 23:44:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
543,543,Maximum Allowable Depletion (MAD) 50%,"Soil Watch 10, PHYTOS 31",50% MAD,3,NaN,NaN,"25 mm for corn, 19 mm for soybeans",1-8,During critical phases,...,NaN,NaN,seed-local:fdf0162d-8971-4c8f-b80a-68d0af3ab55e,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,seed-local:fdf0162d-8971-4c8f-b80a-68d0af3ab55e,[{'title': 'Understanding the impact of irriga...,2025-09-12 23:44:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
544,544,Maximum Allowable Depletion (MAD) 30%,"Soil Watch 10, PHYTOS 31",30% MAD,3,NaN,NaN,"32 mm for corn, 25 mm for soybeans",1-8,During critical phases,...,NaN,NaN,seed-local:fdf0162d-8971-4c8f-b80a-68d0af3a

In [ ]:
# including txt files
info_txt = []
for col in all_samples.columns:
  valid = all_samples.loc[(all_samples[col] != "NA") & (all_samples[col].notna())]
  info_txt.append({
      'feature': col,
      'total_samples': len(all_samples[col]),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(all_samples[col]) * 100.0)
  })
info_txt = pd.DataFrame(info_txt)
info_txt = info_txt.loc[info_txt['percentage'] < 100]
info_txt

,feature,total_samples,valid_samples,percentage
1,irrigation_scheduling_method,547,490,89.579525
2,sensor_type,547,99,18.098720
3,sensor_threshold,547,53,9.689214
4,number_of_sensors,547,46,8.409506
5,evapotranspiration_level,547,213,38.939671
6,daily_drainage,547,10,1.828154
7,amount_irrigated,547,376,68.738574
8,number_of_irrigation_events,547,178,32.541133
9,irrigation_time,547,213,38.939671
10,location,547,506,92.504570


In [ ]:
# only complete pdfs files
info = []
for col in all_samples.columns:
  valid = all_samples.loc[(all_samples[col] != "NA") & (all_samples[col].notna()) & (all_samples["content source"] == "pdf")]
  total_pdf = all_samples.loc[all_samples["content source"] == "pdf"]
  info.append({
      'feature': col,
      'total_samples': len(total_pdf),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(total_pdf)) * 100.0
  })
info = pd.DataFrame(info)
info = info.loc[info['percentage'] < 100]
info

,feature,total_samples,valid_samples,percentage
1,irrigation_scheduling_method,413,377,91.283293
2,sensor_type,413,66,15.980630
3,sensor_threshold,413,29,7.021792
4,number_of_sensors,413,13,3.147700
5,evapotranspiration_level,413,155,37.530266
6,daily_drainage,413,3,0.726392
7,amount_irrigated,413,322,77.966102
8,number_of_irrigation_events,413,142,34.382567
9,irrigation_time,413,167,40.435835
10,location,413,396,95.883777


In [ ]:
info.sort_values(by= "percentage", ascending= False, inplace= True)
info

,feature,total_samples,valid_samples,percentage
11,plant_type,413,411,99.515738
10,location,413,396,95.883777
1,irrigation_scheduling_method,413,377,91.283293
17,plant_yield,413,366,88.619855
7,amount_irrigated,413,322,77.966102
19,water_productivity,413,315,76.271186
12,number_of_plants,413,229,55.447942
9,irrigation_time,413,167,40.435835
5,evapotranspiration_level,413,155,37.530266
8,number_of_irrigation_events,413,142,34.382567


# Data Per Feature (Test 2)

In [ ]:
all_samples = pd.read_csv('/content/drive/MyDrive/1:1_Sruthi_Manoj/Dataset/csv_files/Extra analysis/all_feature_samples (2).csv')

In [ ]:
all_samples.loc[all_samples["content source"] == "pdf"]

,Unnamed: 0,irrigation_scheduling_method,sensor_type,sensor_threshold,number_of_sensors,evapotranspiration_level,daily_drainage,amount_irrigated,number_of_irrigation_events,irrigation_time,...,plant_height,water_productivity,doi_url,file_path,doi,references,processed_at,content source,relevance_score,relevance_dict
0,0,Soil moisture sensor-based method (T-VWC),Soil moisture sensor,VWC max = 0.507 m3 m−3,1,"ETd = 4.63 mm (beginning), 3.79 mm (middle), 3...",0.34-0.43 L plant−1,"1.70-2.00 L plant−1 (beginning), 2.00 L plant−...",12,07:00–18:00,...,NaN,17.37 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
1,1,Accumulated radiation combined with soil moist...,Soil moisture sensor,Accumulated radiation = 0.8 MJ m−2,1,"ETd = 4.66 mm (beginning), 3.03 mm (middle), 1...","0.40 L plant−1 (beginning), 0.34 L plant−1 (mi...","2.03 L plant−1 (beginning), 1.40 L plant−1 (mi...","14 (beginning), 9 (middle), 5 (end)",NaN,...,NaN,20.52 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
2,2,Crop evapotranspiration estimated method using...,NaN,ETc h ≥ 0.4 mm,NaN,"ETd = 2.94 mm (beginning), 2.17 mm (middle), 2...","0.26 L plant−1 (beginning), 0.36 L plant−1 (mi...","1.29 L plant−1 (beginning), 1.12 L plant−1 (mi...","9 (beginning), 9 (middle), 10 (end)",NaN,...,NaN,25.07 kg m−3,https://doi.org/10.1038/s41598-024-52972-x,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1038/s41598-024-52972-x,[{'title': 'Effects of irrigation scheduling o...,2025-09-12 01:42:16,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
3,3,Soil water balance method,"Neutron probe, gravimetric method",50% depletion of total available soil moisture,NaN,"100% ETc, 85% ETc, 70% ETc, 55% ETc, 40% ETc",NaN,"417.2 mm (100% ETc), 341 mm (85% ETc), 286 mm ...","14 (2021/2022), 11 (2022/2023)",December to March (2021/2022 and 2022/2023),...,NaN,1.42 kg m−3 (70% ETc),https://doi.org/10.1177/11786221241274019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1177/11786221241274019,[{'title': 'Effect of Deficit Irrigation on Wh...,2025-09-12 04:01:02,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
4,4,NaN,Neutron probe,NaN,NaN,2000 mm year-1,NaN,"753 mm in 2017, 700 mm in 2018",NaN,Irrigation started one month after sowing,...,NaN,"IWUEhc: 1.97 kg m-3 (S-75), 1.65 kg m-3 (S-150...",https://doi.org/10.1016/j.agwat.2006.03.019,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.1016/j.agwat.2006.03.019,[{'title': 'The Response of Two Drip-Irrigated...,2025-09-12 04:03:56,pdf,8,"[{'relevance_score': 8, 'score_justification':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,579,Deficit Irrigation (Soil Texture: Loam),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,13.51%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,[{'title': 'Evaluating the Effect of Deficit I...,2025-09-13 00:00:24,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
580,580,Deficit Irrigation (Soil Texture: Sandy),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.83%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Manoj/Datase...,https://doi.org/10.3390/plants13050640,[{'title': 'Evaluating the Effect of Deficit I...,2025-09-13 00:00:24,pdf,9,"[{'relevance_score': 9, 'score_justification':..."
581,581,Deficit Irrigation (Soil Available Nitrogen Co...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,3.11%,https://doi.org/10.3390/plants13050640,/content/drive/MyDrive/1:1_Sruthi_Ma

In [ ]:
# including txt files
info_txt = []
for col in all_samples.columns:
  valid = all_samples.loc[(all_samples[col] != "NA") & (all_samples[col].notna())]
  info_txt.append({
      'feature': col,
      'total_samples': len(all_samples[col]),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(all_samples[col]) * 100.0)
  })
info_txt = pd.DataFrame(info_txt)
info_txt = info_txt.loc[info_txt['percentage'] < 100]
info_txt

,feature,total_samples,valid_samples,percentage
1,irrigation_scheduling_method,605,493,81.487603
2,sensor_type,605,129,21.322314
3,sensor_threshold,605,84,13.884298
4,number_of_sensors,605,62,10.247934
5,evapotranspiration_level,605,256,42.314050
6,daily_drainage,605,39,6.446281
7,amount_irrigated,605,333,55.041322
8,number_of_irrigation_events,605,177,29.256198
9,irrigation_time,605,204,33.719008
10,location,605,548,90.578512


In [ ]:
# only complete pdfs files
info = []
for col in all_samples.columns:
  valid = all_samples.loc[(all_samples[col] != "NA") & (all_samples[col].notna()) & (all_samples["content source"] == "pdf")]
  total_pdf = all_samples.loc[all_samples["content source"] == "pdf"]
  info.append({
      'feature': col,
      'total_samples': len(total_pdf),
      'valid_samples': len(valid),
      'percentage': float(len(valid)/len(total_pdf)) * 100.0
  })
info = pd.DataFrame(info)
info = info.loc[info['percentage'] < 100]
info

,feature,total_samples,valid_samples,percentage
1,irrigation_scheduling_method,434,356,82.027650
2,sensor_type,434,89,20.506912
3,sensor_threshold,434,43,9.907834
4,number_of_sensors,434,23,5.299539
5,evapotranspiration_level,434,172,39.631336
6,daily_drainage,434,3,0.691244
7,amount_irrigated,434,249,57.373272
8,number_of_irrigation_events,434,121,27.880184
9,irrigation_time,434,141,32.488479
10,location,434,408,94.009217


In [ ]:
info.sort_values(by= "percentage", ascending= False, inplace= True)
info

,feature,total_samples,valid_samples,percentage
16,plant_type,434,421,97.004608
10,location,434,408,94.009217
1,irrigation_scheduling_method,434,356,82.027650
22,plant_yield,434,338,77.880184
24,water_productivity,434,257,59.216590
7,amount_irrigated,434,249,57.373272
11,latitude,434,226,52.073733
12,longitude,434,226,52.073733
13,altitude,434,176,40.552995
17,number_of_plants,434,175,40.322581
